<a href="https://colab.research.google.com/github/Nuwantha97/Sinhala_spell_and_grammer_checker/blob/Notebooks/Spell_checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Edit Distance (Levenshtein Distance)


The Levenshtein library in Python is a specialized tool for computing Levenshtein distances

- Levenshtein Distance: Calculates the minimum edit distance between two strings.
- Levenshtein Similarity: Measures how similar two strings are, typically on a scale from 0 to 1.
- Other Metrics:
 - Ratio: A normalized version of the distance (1 - distance/max length).
 - Hamming Distance: Number of positions where two strings of equal length differ.
 - Jaro-Winkler Similarity: A more nuanced similarity metric, especially for short strings.

In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.7 MB/s eta 0:00:00


In [ ]:
# Function to load the dictionary from a text file
def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        dictionary = [line.strip() for line in file]
    return dictionary

In [ ]:
import Levenshtein

# Load the dictionary from the text file
sinhala_dictionary = load_dictionary('/content/drive/MyDrive/Projects/Sinhala Spell and Grammer checker/sinhala_full_word_list_2016-10-08.txt')

# Function to find the closest match
def spell_check(word, dictionary):
    # Find the word with the minimum distance
    closest_word = None
    min_distance = float('inf')

    for correct_word in dictionary:
        distance = Levenshtein.distance(word, correct_word)
        if distance < min_distance:
            min_distance = distance
            closest_word = correct_word

    return closest_word, min_distance

# Input word
input_word = "අදනවා"

# Perform spell check
correct_word, distance = spell_check(input_word, sinhala_dictionary)

print(f"Input Word: {input_word}")
print(f"Suggested Correction: {correct_word}")
print(f"Levenshtein Distance: {distance}")


Input Word: අදනවා
Suggested Correction: අඬනවා
Levenshtein Distance: 1
